In [1]:
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
import seaborn as sns 
import matplotlib as plt 
%matplotlib inline
import glob
import csv
import tarfile
import numpy as np 
from tqdm import tqdm
import json
from progressive.bar import Bar
import pickle as pkl
import networkx as nx
from __future__ import unicode_literals

In [2]:
# with open('Bipartite_Drug_graph_Normalized.pkl','rb') as f:
#     drug_association_graph  = pkl.load(f)
    
with open('Bipartite_Drug_graph.pkl','rb') as f:
    drug_association_graph  = pkl.load(f)

with open('Bipartite_Drug_category_graph.pkl','rb') as f:
    drug_cat_association_graph  = pkl.load(f)

In [260]:
len(drug_association_graph.edges()), len(drug_cat_association_graph.edges())

(7954, 33493)

In [3]:
drug_cat_association_graph.nodes(data=True)

[(u'Vitamins', {'Id': u'/categories/DBCAT000230', 'type': 'category'}),
 (u'Prostacyclin Analogues',
  {'Id': u'/categories/DBCAT002688', 'type': 'category'}),
 ('Pravastatin Sodium', {'Id': '0212000X0', 'type': 'drug'}),
 (u'UGT1A1 Substrates',
  {'Id': u'/categories/DBCAT003871', 'type': 'category'}),
 ('Clindamycin/Tretinoin', {'Id': '1306010AB', 'type': 'drug'}),
 (u'Lactates', {'Id': u'/categories/DBCAT001522', 'type': 'category'}),
 (u'Caffeine', {'Id': u'/categories/DBCAT004728', 'type': 'category'}),
 (u'BCRP/ABCG2 Inhibitors',
  {'Id': u'/categories/DBCAT002662', 'type': 'category'}),
 ('Oxycodone Hydrochloride', {'Id': '0407020AD', 'type': 'drug'}),
 (u'Benzoquinones', {'Id': u'/categories/DBCAT001600', 'type': 'category'}),
 ('Floxuridine', {'Id': '0801030G0', 'type': 'drug'}),
 (u'Cytochrome P-450 CYP2B6 Inhibitors (strength unknown)',
  {'Id': u'/categories/DBCAT004517', 'type': 'category'}),
 (u'Sucrose-specific Enzyme',
  {'Id': u'/categories/DBCAT003477', 'type': 'categ

In [4]:
files = glob.glob('../BL_Work/openPrescribe/uncompressed/*.tar.gz')
print files

[u'../BL_Work/openPrescribe/uncompressed/T201010PDPI BNFT.CSV.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201902PDPI BNFT.csv.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201902CHEM SUBS.csv.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201008ADDR BNFT.CSV.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201601CHEM SUBS.csv.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201601PDPI BNFT.csv.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201011PDPI BNFT.CSV.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201901ADDR BNFT.csv.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201009PDPI BNFT.CSV.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201011ADDR BNFT.CSV.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201010ADDR BNFT.CSV.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201010CHEM SUBS.CSV.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201602ADDR BNFT.csv.tar.gz', u'../BL_Work/openPrescribe/uncompressed/T201603PDPI BNFT.CSV.tar.gz', u'../BL_Work/openPr

In [399]:
#Different functions to extract different data form the prescription 

def cleanStringofUTF(string):
    cleaned = string.replace('\xe8','e').replace('\xf6','o')
    return cleaned

def enrichdrugs(chem_dict , drugs):
    diabetes_drug_words = [drugs[k]['name'].lower() for k in drugs]
    for drug in chem_dict:
        Name = chem_dict[drug]['name'].replace('(','').replace(')','')
        slot1 = Name.lower().split('/')
        slot2 = Name.lower().split(' ')
        slot3 = Name.lower().split(' & ')
        common1 = set(diabetes_drug_words).intersection(slot1)
        common2 = set(diabetes_drug_words).intersection(slot2)
        common3 = set(diabetes_drug_words).intersection(slot3)
        
        if len(common1) > 0 or len(common2) > 0 or len(common3) > 0:
            print common1 , common2 , common3
            drugs[chem_dict[drug]['code']] = {'disease':'' , 'disease_given_drug':0.0 , 'matched_disease':'', 'name':chem_dict[drug]['name'].strip() }

            
            
def makeChemDict(BNF_Chem):
    chem_dict = {}
    for index, row in BNF_Chem.iterrows():
        chem_dict[row['UNII_drugbank']] = {}
        chem_dict[row['UNII_drugbank']]['name'] = row['NAME']
        chem_dict[row['UNII_drugbank']]['code'] = row['BNF_code']
    return chem_dict
    
def getDrugCategory(categorylist, BNF_Chem, drugbankDict):
    allMatched = []
    drugs = {}
    chem_dict = makeChemDict(BNF_Chem)
    
    for k in drugbankDict:
        if len(drugbankDict[k]['Categories']) > 0:
            for cat in drugbankDict[k]['Categories']:
                matched_memo = []
                catString = cat.values()[0]#.split('\u2014')[-1]
                t = catString.lower().strip()
                for categoryString in categorylist:
                    categoryString = categoryString.lower()
                    if t.find(categoryString) >= 0:
                        matched_memo.append(categoryString)
                if k in chem_dict:
                    if len(matched_memo) > 0:# == len(categorylist):
                        allMatched.append(k)
                        print chem_dict[k]
                        drugs[chem_dict[k]['code']] = {}
                        drugs[chem_dict[k]['code']]['name'] = chem_dict[k]['name']
                        drugs[chem_dict[k]['code']]['matched_cat'] = categorylist
    enrichdrugs(chem_dict,drugs)               
    return list(set(allMatched)) , drugs


def getDrugforDiseaseDrugbank(categorylist, BNF_Chem, drugbankDict):
    allMatched = []
    drugs = {}
    chem_dict = makeChemDict(BNF_Chem)
    
    for k in drugbankDict:
        if len(drugbankDict[k]['Associations']) > 0:
            for cat in drugbankDict[k]['Associations']:
                matched_memo = []
                catString = cat.values()[0]
                t = catString.lower().strip()
                for categoryString in categorylist:
                    categoryString = categoryString.lower()
                    if t.find(categoryString) >= 0:
                        matched_memo.append(categoryString)
                if k in chem_dict:
                    if len(matched_memo) > 0:
                        allMatched.append(k)
                        print chem_dict[k]
                        drugs[chem_dict[k]['code']] = {}
                        drugs[chem_dict[k]['code']]['name'] = chem_dict[k]['name']
                        drugs[chem_dict[k]['code']]['matched_cat'] = categorylist
    enrichdrugs(chem_dict,drugs)               
    return  drugs


def findDrugsForDisease(Graph, Disease, BNF_Chem ):#,threshProb):
    chem_dict = makeChemDict(BNF_Chem)
    drugs = {}
    for e in Graph.edges(data=True):
        if (cleanStringofUTF(e[1]).lower().find(Disease.lower()) >=0) or (cleanStringofUTF(e[0]).lower().find(Disease.lower()) >= 0) :
            drugNode = ''
            matchedDisease = ''
            if Graph.node[e[0]]['type'] == 'symptom':
                drugNode = e[1]
                matchedDisease = e[0]
            else:
                drugNode = e[0]
                matchedDisease = e[1]
            drugs[Graph.node[drugNode]['Id']] = {}
            drugs[Graph.node[drugNode]['Id']]['name'] = drugNode
            drugs[Graph.node[drugNode]['Id']]['matched_disease'] = matchedDisease
            drugs[Graph.node[drugNode]['Id']]['disease'] = Disease
    enrichdrugs(chem_dict,drugs)
    return drugs




def findDrugsForCategory(Graph, Cat, BNF_Chem ):#,threshProb):
    chem_dict = makeChemDict(BNF_Chem)
    drugs = {}
    for e in Graph.edges(data=True):
        if (cleanStringofUTF(e[1]).lower().find(Cat.lower()) >=0) or (cleanStringofUTF(e[0]).lower().find(Cat.lower()) >= 0) :
            drugNode = ''
            matchedDisease = ''
            if Graph.node[e[0]]['type'] == 'category':
                drugNode = e[1]
                matchedDisease = e[0]
            else:
                drugNode = e[0]
                matchedDisease = e[1]
            print Graph.node[drugNode]['Id']
            drugs[Graph.node[drugNode]['Id']] = {}
            drugs[Graph.node[drugNode]['Id']]['name'] = drugNode
            drugs[Graph.node[drugNode]['Id']]['matched_cat'] = matchedDisease
            drugs[Graph.node[drugNode]['Id']]['category'] = Cat
    enrichdrugs(chem_dict,drugs)
    return drugs

def findDrugByName(Graph, name):
    foundDrugs = []
    for node in Graph.nodes(data=True):
        if node[1]['type'] == 'drug':
            if name.lower() in node[0].lower().split(' '):
                foundDrugs.append(node)
    return foundDrugs

def relabelNodes(graph):
    mapping = {}
    for node in graph.nodes():
        mapping[node] = cleanStringofUTF(node)
        
    return nx.relabel_nodes(graph,mapping)


def readChem(year,files):
    for f in files:
        if f.find(year)>0:
            if f.find('CHEM') > 0:
                df = pd.read_csv(f,compression='infer',header=None,skiprows=1)
                return df

def readAddress(year,files):
    for f in files:
        if f.find(year)>0:
            if f.find('ADDR') > 0:
                df = pd.read_csv(f,compression='infer',header=None,skiprows=1)
                return df

def readPrescriptions(year,files):
    for f in files:
        if f.find(year)>0:
            if f.find('PDPI') > 0:
                df = pd.read_csv(f,compression='infer',header=None,skiprows=1)
                return df



In [14]:
# # chem = readChem('201601',files)
# chem = pd.read_csv('../BL_Work/CHEM_MASTER_MAP.csv')
# len(chem)

# # chem = chem.dropna()

In [15]:
drugbank_dict = json.load(open('Drugbank_drugs_data.json','rb'))

In [16]:
drugbank_dict['WK2XYI10QM']

{u'ATC_Codes': [{u'/atc/G02CC01': u'G02CC01 \u2014 Ibuprofen'},
  {u'/atc/G02CC': u'G02CC \u2014 Antiinflammatory products for vaginal administration'},
  {u'/atc/G02C': u'G02C \u2014 OTHER GYNECOLOGICALS'},
  {u'/atc/G02': u'G02 \u2014 OTHER GYNECOLOGICALS'},
  {u'/atc/G': u'G \u2014 GENITO URINARY SYSTEM AND SEX HORMONES'},
  {u'/atc/C01EB16': u'C01EB16 \u2014 Ibuprofen'},
  {u'/atc/C01EB': u'C01EB \u2014 Other cardiac preparations'},
  {u'/atc/C01E': u'C01E \u2014 OTHER CARDIAC PREPARATIONS'},
  {u'/atc/C01': u'C01 \u2014 CARDIAC THERAPY'},
  {u'/atc/C': u'C \u2014 CARDIOVASCULAR SYSTEM'},
  {u'/atc/M01AE01': u'M01AE01 \u2014 Ibuprofen'},
  {u'/atc/M01AE': u'M01AE \u2014 Propionic acid derivatives'},
  {u'/atc/M01A': u'M01A \u2014 ANTIINFLAMMATORY AND ANTIRHEUMATIC PRODUCTS, NON-STEROIDS'},
  {u'/atc/M01': u'M01 \u2014 ANTIINFLAMMATORY AND ANTIRHEUMATIC PRODUCTS'},
  {u'/atc/M': u'M \u2014 MUSCULO-SKELETAL SYSTEM'},
  {u'/atc/M02AA13': u'M02AA13 \u2014 Ibuprofen'},
  {u'/atc/M02AA':

In [150]:
chem = pd.read_csv('../BL_Work/CHEM_MASTER_MAP.csv')

In [151]:
chem.head()

,Unnamed: 0,BNF_code,NAME,UNII_drugbank
0,0,0101010A0,Alexitol Sodium,[]
1,1,0101010B0,Almasilate,[]
2,2,0101010C0,Aluminium Hydroxide,[]
3,3,0101010D0,Aluminium Hydroxide With Magnesium,[]
4,4,0101010E0,Hydrotalcite,17432CG1KU


In [411]:
opiod_matched , drugs_opiates = getDrugCategory(['opiate'],chem,drugbank_dict)

{u'code': '0407020K0', u'name': 'Diamorphine Hydrochloride (Systemic)                        '}
{u'code': '0410030A0', u'name': 'Buprenorphine Hydrochloride                                 '}
{u'code': '0410030A0', u'name': 'Buprenorphine Hydrochloride                                 '}
{u'code': '0410030A0', u'name': 'Buprenorphine Hydrochloride                                 '}
{u'code': '0106060A0', u'name': 'Methylnaltrexone Bromide                                    '}
{u'code': '0704050G0', u'name': 'Apomorphine Hydrochloride                                   '}
{u'code': '0407020C0', u'name': 'Codeine Phosphate                                           '}
{u'code': '0407020C0', u'name': 'Codeine Phosphate                                           '}
{u'code': '030902030', u'name': 'Pholcodine                                                  '}
{u'code': '0407020U0', u'name': 'Pentazocine Lactate                                         '}
{u'code': '0407020U0', u'name': 'Pentazo

In [550]:
antiBio_matched , drugs_antiBio = getDrugCategory(['antibiotic','Anti-Bacterial Agents'],chem,drugbank_dict)

{u'code': '0504050T0', u'name': 'Paromomycin Sulfate                                         '}
{u'code': '0802020U0', u'name': 'Sirolimus                                                   '}
{u'code': '0802020U0', u'name': 'Sirolimus                                                   '}
{u'code': '1103010Z0', u'name': 'Vancomycin Hydrochloride                                    '}
{u'code': '1310012C0', u'name': 'Chlortetracycline Hydrochloride                             '}
{u'code': '1310012C0', u'name': 'Chlortetracycline Hydrochloride                             '}
{u'code': '050102020', u'name': 'Cefpirome Sulfate                                           '}
{u'code': '0501012U0', u'name': 'Temocillin Sodium                                           '}
{u'code': '0501120Y0', u'name': 'Moxifloxacin                                                '}
{u'code': '0501015P0', u'name': 'Pivmecillinam Hydrochloride                                 '}
{u'code': '0501050S0', u'name': 'Spiramy

In [623]:
antiDM_matched , drugs_antiDM = getDrugCategory(['diabetes','Type 2'],chem,drugbank_dict)

{u'code': '0601023I0', u'name': 'Guar Gum                                                    '}
{u'code': '0601022B0', u'name': 'Metformin Hydrochloride                                     '}
{u'code': '0601023AQ', u'name': 'Dulaglutide                                                 '}
{u'code': '0601021X0', u'name': 'Tolbutamide                                                 '}
{u'code': '0601021X0', u'name': 'Tolbutamide                                                 '}
{u'code': '0601011P0', u'name': 'Insulin Glulisine                                           '}
{u'code': '0601023AB', u'name': 'Liraglutide                                                 '}
{u'code': '0601021A0', u'name': 'Glimepiride                                                 '}
{u'code': '0205052R0', u'name': 'Telmisartan with Diuretic                                   '}
{u'code': '0601012X0', u'name': 'Insulin Detemir                                             '}
{u'code': '0601021R0', u'name': 'Gliquid

In [624]:
drugs_opiates

{'0106060A0': {u'disease': u'',
  u'disease_given_drug': 0.0,
  u'matched_disease': u'',
  u'name': 'Methylnaltrexone Bromide'},
 '0106060B0': {u'disease': u'',
  u'disease_given_drug': 0.0,
  u'matched_disease': u'',
  u'name': 'Naloxegol'},
 '0309010U0': {u'disease': u'',
  u'disease_given_drug': 0.0,
  u'matched_disease': u'',
  u'name': 'Noscapine'},
 '030902030': {u'disease': u'',
  u'disease_given_drug': 0.0,
  u'matched_disease': u'',
  u'name': 'Pholcodine'},
 '0309020R0': {u'disease': u'',
  u'disease_given_drug': 0.0,
  u'matched_disease': u'',
  u'name': "Dextromethorphan Hydrobrom Comp Prep's"},
 '040702010': {u'disease': u'',
  u'disease_given_drug': 0.0,
  u'matched_disease': u'',
  u'name': 'Morphine Anhydrous'},
 '040702040': {u'disease': u'',
  u'disease_given_drug': 0.0,
  u'matched_disease': u'',
  u'name': 'Tramadol Hydrochloride'},
 '040702050': {u'disease': u'',
  u'disease_given_drug': 0.0,
  u'matched_disease': u'',
  u'name': 'Hydromorphone Hydrochloride'},
 '0

In [552]:
# chem.head(n=100)

In [553]:
opiod_BNF = chem[chem['UNII_drugbank'].isin(opiod_matched)]

In [554]:
antiBio_BNF = chem[chem['UNII_drugbank'].isin(antiBio_matched)]

In [625]:
antiDM_BNF = chem[chem['UNII_drugbank'].isin(antiDM_matched)]

In [555]:
len(antiBio_BNF)

252

In [540]:
# antiBio_BNF.head(n=100)

In [415]:
pd.set_option('display.max_columns', None)  
len(opiod_BNF)

54

In [350]:
# name_Key_Dict = {k : drugbank_dict[k]['Name'] for k in drugbank_dict}
# for index , row in chem.iterrows():
#     for k in name_Key_Dict:
#         drug = row[1]
#         slot1 =[word.lower().strip() for word in  drug.lower().split('/')]
#         slot2 =[word.lower().strip() for word in  drug.lower().split(' ')]
#         slot3 =[word.lower().strip() for word in drug.lower().split('&')]
#         if name_Key_Dict[k].lower().strip() in slot1 or name_Key_Dict[k].lower().strip() in slot2 or name_Key_Dict[k].lower().strip() in slot3:
#             row[2] = k
            

In [351]:
# name_Key_Dict

In [352]:
# chem.to_csv('../BL_Work/CHEM_MASTER_MAP.csv')

In [353]:
drug_association_graph = relabelNodes(drug_association_graph)

In [354]:
# chem.rename(columns={0:'BNF_code',
#                           1:'NAME',
#                           2:'UNII_drugbank'}, 
#                  inplace=True)

In [355]:
# drug_association_graph.nodes(data=True)

In [356]:
len([k for k in drug_association_graph.nodes(data=True) if k[1]['type'] == 'symptom'])

2662

In [357]:
# drug_cat_association_graph.nodes(data=True)

In [358]:
# print [node[1]['Id'] for node in drug_cat_association_graph.nodes(data=True) if node[1]['type'] == 'drug']

In [359]:
# db_opioid = findDrugsForCategory(drug_cat_association_graph,"Opioid",chem )

In [360]:
# db_opioid.keys()

In [361]:
# db_opioid

In [417]:
opiod_manual = {'0407010A0':'Combination',
'0407010F0':'Combination',
'0407010M0':'Combination',
'0407010M0':'Combination',
'0407010N0':'Combination',
'0407010N0':'Combination',
'0407010Q0':'Combination',
'0407010W0':'Combination',
'0407010X0':'Combination',
'040702020':'Morphine',
'040702040':'Tramadol',
'040702050':'Hydromorphone',
'0407020A0':'Fentanyl',
'0407020AB':'Papaveretum',
'0407020AD':'Oxycodone',
'0407020AF':'Combination',
'0407020AG':'Tapentadol',
'0407020B0':'Buprenorphine',
'0407020C0':'Codeine',
'0407020D0':'Dextromoramide ',
'0407020E0':'Dextropropoxyphene',
'0407020G0':'Dihydrocodeine ',
'0407020H0':'Dipipanone ',
'0407020K0':'Diamorphine ',
'0407020L0':'Meptazinol ',
'0407020M0':'Methadone ',
'0407020P0':'Morphine ',
'0407020Q0':'Morphine ',
'0407020T0':'Pentazocine ',
'0407020U0':'Pentazocine ',
'0407020V0':'Pethidine ',
'0407020W0':'Opium',
'0407020X0':'Phenazocine ',
'0407020Y0':'Nalbuphine ',
'0407020Z0':'Oxycodone',
'1001010J0':'Combination',}

In [418]:
len(opiod_manual)

34

In [419]:
 len(set(opiod_manual.keys()).intersection(opiod_BNF['BNF_code']))

19

In [420]:
len(opiod_BNF)

54

In [421]:
opiod_BNF

,Unnamed: 0,BNF_code,NAME,UNII_drugbank
59,59,010200010,Papaverine Hydrochloride,DAA13NKG2Q
88,88,0104020D0,Codeine Phosphate Compound Mixtures,UX6OWY2V7J
91,91,0104020N0,Opium & Morphine,76I7G6D29C
154,154,0106060A0,Methylnaltrexone Bromide,0RK7M7IABE
155,155,0106060B0,Naloxegol,44T7335BKE
691,691,0309010C0,Codeine Phosphate,UX6OWY2V7J
692,692,0309010L0,Dextromethorphan Hydrobromide,7355X3ROTS
693,693,0309010N0,Diamorphine Hydrochloride,70D95007SX
695,695,0309010S0,Methadone Hydrochloride,UC6VBE7V1Z
696,696,0309010U0,Noscapine,8V32U4AOQU


In [702]:
import re 
s = "Doxycycline Hyclate_Tab Disper 100mg S/F"
array = re.findall(r'[0-9]+', s) 
print array

[u'100']


In [680]:
pdp.head()

,0,1,2,3,4,5,6,7,8,9,10
0,Q44,RJN,Y05218,0204000H0AAAJAJ,Bisoprolol Fumar_Tab 2.5mg,1.0,0.96,0.90,28.0,201602.0,
1,Q44,RJN,Y05218,0205051R0AAADAD,Ramipril_Cap 10mg,1.0,1.13,1.06,28.0,201602.0,
2,Q44,RJN,Y05218,0212000B0AAABAB,Atorvastatin_Tab 20mg,1.0,1.21,1.13,28.0,201602.0,
3,Q44,RJN,Y05218,0501030I0AAAEAE,Doxycycline Hyclate_Tab Disper 100mg S/F,1.0,6.14,5.79,10.0,201602.0,
4,Q44,RJN,Y05218,0601022B0BBACAS,Glucophage SR_Tab 500mg,1.0,15.96,14.78,168.0,201602.0,


In [ ]:
pdp[pdp[2].isin(opiod_BNF['UNII_drugbank'])]

In [422]:
missed= [k for k in opiod_manual if k not in list(opiod_BNF['BNF_code'])]

In [424]:
extra = [k for k in list(opiod_BNF['BNF_code']) if k not in opiod_manual]

In [798]:
# opiod_BNF['NAME']

In [449]:
chem[chem['BNF_code'].isin(extra)]['NAME']

59      Papaverine Hydrochloride                                    
88      Codeine Phosphate Compound Mixtures                         
91      Opium & Morphine                                            
154     Methylnaltrexone Bromide                                    
155     Naloxegol                                                   
691     Codeine Phosphate                                           
692     Dextromethorphan Hydrobromide                               
693     Diamorphine Hydrochloride                                   
695     Methadone Hydrochloride                                     
696     Noscapine                                                   
697     Pholcodine                                                  
700     Diphenhydramine HCl/Pholcodine                              
701     Guaiacol & Codeine                                          
718     Powdered Opium                                              
721     Dextromethorphan Hydrobrom

In [244]:
db_diabetes = findDrugsForDisease(drug_association_graph,"diabetes",chem )

set([]) set([u'dulaglutide']) set([])
set([]) set([u'hydrochlorothiazide']) set([])
set([]) set([u'tolbutamide']) set([])
set([]) set([u'liraglutide']) set([])
set([]) set([u'glimepiride']) set([])
set([]) set([u'tolazamide']) set([])
set([u'dapagliflozin']) set([u'dapagliflozin/metformin']) set([])
set([]) set([u'lixisenatide']) set([])
set([u'empagliflozin']) set([u'empagliflozin/metformin']) set([])
set([]) set([u'nateglinide']) set([])
set([]) set([u'sitagliptin']) set([])
set([]) set([u'albiglutide']) set([])
set([]) set([u'chlorpropamide']) set([])
set([]) set([u'miglitol']) set([])
set([]) set([u'alogliptin']) set([])
set([u'metformin hydrochloride']) set([]) set([])
set([]) set([u'glipizide']) set([])
set([]) set([u'repaglinide']) set([])
set([]) set([u'acarbose']) set([])
set([]) set([u'bromocriptine']) set([])
set([u'metformin hydrochloride']) set([]) set([])
set([u'linagliptin']) set([u'linagliptin/metformin']) set([])
set([u'canagliflozin']) set([u'canagliflozin/metformin']

In [261]:
# db_diabetes_drugBank = getDrugforDiseaseDrugbank(['diabetes'],chem,drugbank_dict)

In [262]:
len(db_diabetes_drugBank)

37

In [263]:
len(db_diabetes)

54

In [264]:
%store -r diabetes_drugs

In [269]:
missed_drugs = [diabetes_drugs[k]['ChemName'] for k in diabetes_drugs if k not in db_diabetes]
print missed_drugs

['Protamine Zinc Insulin', 'Urine Testing Reagents', 'Insulin Zinc Suspension', 'Soluble Insulin (Neutral Insulin)', 'Guar Gum', 'Other Screening & Monitoring Agent Preps', 'Glucose Blood Testing Reagents', 'Biphasic Insulin Lispro', 'Biphasic Isophane Insulin', 'Biphasic Insulin Aspart', 'Ins Degludec/Liraglutide', 'Ketone Blood Testing Reagents']


In [232]:
# db_diabetes.keys()

In [253]:
# [k for k in diabetes_drugs if k not in db_diabetes]

In [254]:
# [k for k in diabetes_drugs if k not in db_diabetes_drugBank]

In [234]:
# diabetes_drugs['0402010AF']

In [238]:
len([db_diabetes[k] for k in db_diabetes if k not in ])

10

In [255]:
diabetes_drugs

{'0601011A0': {'BNF_Classification': '6.1.1',
  'ChemName': 'Insulin Aspart',
  'category': 'Insulin',
  'disease': 'diabetes'},
 '0601011L0': {'BNF_Classification': '6.1.1',
  'ChemName': 'Insulin Lispro',
  'category': 'Insulin',
  'disease': 'diabetes'},
 '0601011N0': {'BNF_Classification': '6.1.1',
  'ChemName': 'Soluble Insulin (Neutral Insulin)',
  'category': 'Insulin',
  'disease': 'diabetes'},
 '0601011P0': {'BNF_Classification': '6.1.1',
  'ChemName': 'Insulin Glulisine',
  'category': 'Insulin',
  'disease': 'diabetes'},
 '0601011R0': {'BNF_Classification': '6.1.1',
  'ChemName': 'Insulin Human',
  'category': 'Insulin',
  'disease': 'diabetes'},
 '0601012D0': {'BNF_Classification': '6.1.1',
  'ChemName': 'Biphasic Isophane Insulin',
  'category': 'Insulin',
  'disease': 'diabetes'},
 '0601012F0': {'BNF_Classification': '6.1.1',
  'ChemName': 'Biphasic Insulin Lispro',
  'category': 'Insulin',
  'disease': 'diabetes'},
 '0601012G0': {'BNF_Classification': '6.1.1',
  'ChemNam

In [256]:
# DiseaseDrugs = json.load(open('../processed/DiseasesMatched.json','rb'))

In [270]:
diseases = ["psoriasis",
 "gastroparesis",
 "multiple sclerosis",
 "crohn's disease",
 "interstitial cystitis",
 "rheumatoid",
 "parkinson",
 "dysautonomia",
 "kidney stones",
 "chronic fatigue syndrome",
 "bipolar",
 "hypothyroidism",
 "sleep apnea",
 "Meniere's disease",
 "dementia",
 "irritable bowel syndrome",
 "depression",
 "diabetes"]

In [470]:
DiseaseDrugs = {}
for d in diseases:
    drugs = findDrugsForDisease(drug_association_graph,d ,chem)
    for drug in drugs:
        DiseaseDrugs[drug] = {}
        DiseaseDrugs[drug]['chemName'] = drugs[drug]['name']
        DiseaseDrugs[drug]['disease'] = d

set([]) set([u'calcipotriol']) set([])
set([]) set([u'dexamethasone']) set([])
set([]) set([u'acitretin']) set([])
set([]) set([u'pimecrolimus']) set([])
set([]) set([u'methylprednisolone']) set([])
set([]) set([u'betamethasone']) set([])
set([]) set([u'tacrolimus']) set([])
set([]) set([u'infliximab']) set([])
set([]) set([u'prednisolone']) set([])
set([]) set([u'desoximetasone']) set([])
set([]) set([u'adalimumab']) set([])
set([]) set([u'tazarotene']) set([])
set([]) set([u'ciclosporin']) set([])
set([]) set([u'etanercept']) set([])
set([]) set([u'prednisone']) set([])
set([]) set([u'apremilast']) set([])
set([]) set([u'calcitriol']) set([])
set([]) set([u'azathioprine']) set([])
set([]) set([u'methoxsalen']) set([])
set([]) set([u'hydrocortisone']) set([])
set([]) set([u'methotrexate']) set([])
set([]) set([u'domperidone']) set([])
set([]) set([u'erythromycin']) set([])
set([]) set([u'daclizumab']) set([])
set([]) set([u'tetracosactide']) set([])
set([]) set([u'alemtuzumab']) set([

In [472]:
diabetes = {k:DiseaseDrugs[k] for k in DiseaseDrugs if DiseaseDrugs[k]['disease'] == 'diabetes'}

In [474]:
diabetes_metformin = {k:DiseaseDrugs[k] for k in DiseaseDrugs if DiseaseDrugs[k]['chemName'].find('Metformin') >= 0}

In [486]:
diabetes_metformin

{'0601022B0': {u'chemName': u'Metformin Hydrochloride',
  u'disease': u'diabetes'},
 '0601023AD': {u'chemName': u'Metformin Hydrochloride/Sitagliptin',
  u'disease': u'diabetes'},
 '0601023AF': {u'chemName': 'Linagliptin/Metformin', u'disease': u'diabetes'},
 '0601023AH': {u'chemName': 'Saxagliptin/Metformin', u'disease': u'diabetes'},
 '0601023AJ': {u'chemName': u'Alogliptin/Metformin', u'disease': u'diabetes'},
 '0601023AL': {u'chemName': 'Dapagliflozin/Metformin',
  u'disease': u'diabetes'},
 '0601023AP': {u'chemName': 'Canagliflozin/Metformin',
  u'disease': u'diabetes'},
 '0601023AR': {u'chemName': 'Empagliflozin/Metformin',
  u'disease': u'diabetes'},
 '0601023V0': {u'chemName': 'Metformin Hydrochloride/Rosiglitazone',
  u'disease': u'diabetes'},
 '0601023W0': {u'chemName': 'Metformin Hydrochloride/Pioglitazone',
  u'disease': u'diabetes'},
 '0601023Z0': {u'chemName': 'Metformin Hydrochloride/Vildagliptin',
  u'disease': u'diabetes'}}

In [434]:
json.dump(DiseaseDrugs,open('Disease_Drug_DrugBank.json','wb'))

In [274]:
SymptomDrugs = json.load(open('SymptomsMatched.json','rb'))

In [275]:
symptoms = [k.replace('+',' ').replace('"','').lower() for k in SymptomDrugs.keys()]
print symptoms

[u'impairment level of vision', u'backache', u'dyssomnia', u'loss of hair', u'mood swings', u'tired', u'weight gain', u'emotional upset', u'disorientated', u'clouded consciousness', u'stomach ache', u'increased thirst', u'sleep deprivation', u'heartburn', u'abdominal pain', u'swelling', u'feeling agitated', u'impairment of balance', u'paresthesia', u'blurring of visual image', u'feeling angry', u'loss of appetite', u'pain', u'tremor', u'rectal hemorrhage', u'abnormal gait', u'impotence', u'eruption of skin', u'feeling hopeless', u'indigestion', u"parkinson's disease", u'trigeminal neuralgia', u'bowel dysfunction', u'parkinsonism', u'delusional disorder', u'muscle pain', u'restlessness', u'malaise', u'thoughts of self harm', u'dementia', u'depressive disorder', u'hypothyroidism', u'reduced mobility', u'jaundice', u'lack of energy', u'joint pain', u'vertigo', u'headache', u'tinnitus', u'decrease in appetite', u'insomnia', u'multiple sclerosis', u'dry skin', u'blood in urine', u'increased

In [277]:
symptoms_matched = {}
for d in symptoms:
    drugs = findDrugsForDisease(drug_association_graph,d, chem)
    for drug in drugs:
        symptoms_matched[drug] = {}
        symptoms_matched[drug]['chemName'] = drugs[drug]['name']
        symptoms_matched[drug]['disease'] = d

set([]) set([u'capsaicin']) set([])
set([]) set([u'tenoxicam']) set([])
set([]) set([u'naproxen']) set([])
set([]) set([u'camphor']) set([])
set([]) set([u'caffeine']) set([])
set([]) set([u'modafinil']) set([])
set([]) set([u'orlistat']) set([])
set([]) set([u'topiramate']) set([])
set([]) set([u'magaldrate']) set([])
set([]) set([u'omeprazole']) set([])
set([]) set([u'cimetidine']) set([])
set([]) set([u'famotidine']) set([])
set([]) set([u'esomeprazole']) set([])
set([]) set([u'paracetamol', u'caffeine']) set([u'paracetamol'])
set([]) set([u'nortriptyline']) set([])
set([]) set([u'capsaicin']) set([])
set([]) set([u'denosumab']) set([])
set([]) set([u'gabapentin']) set([])
set([]) set([u'celecoxib']) set([])
set([]) set([u'baclofen']) set([])
set([]) set([u'dextropropoxyphene']) set([])
set([]) set([u'buprenorphine']) set([])
set([]) set([u'bromfenac']) set([])
set([]) set([u'oxandrolone']) set([])
set([]) set([u'aspirin']) set([])
set([]) set([u'tolmetin']) set([])
set([]) set([u't

set([]) set([u'adrenaline']) set([])
set([]) set([u'droperidol']) set([])
set([]) set([u'fructose']) set([])
set([]) set([u'lorazepam']) set([])
set([]) set([u'aprepitant']) set([])
set([]) set([u'granisetron']) set([])
set([]) set([u'perphenazine']) set([])
set([]) set([u'nabilone']) set([])
set([u'carbidopa']) set([]) set([])
set([]) set([u'haloperidol']) set([])
set([]) set([u'mirtazapine']) set([])
set([]) set([u'clozapine']) set([])
set([]) set([u'paracetamol']) set([u'paracetamol'])
set([]) set([u'tetracosactide']) set([])
set([]) set([u'aspirin']) set([])
set([u'ciprofloxacin']) set([]) set([])
set([]) set([u'aspirin']) set([])
set([]) set([u'naproxen']) set([])
set([]) set([u'sulfadiazine']) set([])
set([]) set([u'tetracycline']) set([])
set([]) set([u'erythromycin']) set([])
set([]) set([u'ibuprofen']) set([])
set([]) set([u'colchicine']) set([])
set([]) set([u'hydrochlorothiazide']) set([])
set([]) set([u'torasemide']) set([])
set([]) set([u'nifedipine']) set([])
set([]) set(

In [426]:
symptoms_matched

{'0101010B0': {u'chemName': u'Almasilate', u'disease': u'indigestion'},
 '0101010C0': {u'chemName': u'Aluminium Hydroxide',
  u'disease': u'indigestion'},
 '0101010F0': {u'chemName': u'Magnesium Carbonate',
  u'disease': u'indigestion'},
 '0101010I0': {u'chemName': u'Magnesium Oxide', u'disease': u'indigestion'},
 '0101010J0': {u'chemName': u'Magnesium Trisilicate',
  u'disease': u'indigestion'},
 '0101010M0': {u'chemName': u'Magaldrate', u'disease': u'indigestion'},
 '0101010S0': {u'chemName': u'Calcium Carbonate & Simeticone',
  u'disease': u'indigestion'},
 '0101010T0': {u'chemName': u'Magnesium Sulphate',
  u'disease': u'constipation'},
 '0101012B0': {u'chemName': u'Sodium Bicarbonate', u'disease': u'diarrhea'},
 '0102000AB': {u'chemName': u'Hyoscyamine Sulfate',
  u'disease': u'irritable bowel syndrome'},
 '0102000AD': {u'chemName': u'Pinaverium Bromide',
  u'disease': u'irritable bowel syndrome'},
 '0102000AG': {u'chemName': u'Tegaserod Maleate', u'disease': u'constipation'},
 '0

In [278]:
json.dump(symptoms_matched,open('Symptom_Drug_DrugBank.json','wb'))

In [279]:
# def tokenizeChemName(name):
#     tokens = []
#     spaces = name.lower().split(' ')
#     for s in spaces:
#         subtoken = s.split('/')
#         tokens+=subtoken
#     return tokens
# print tokenizeChemName("aLsaSdjaklsdj/askldjals asdad")

# def looslyMatch(list1 , list2):
#     if len(set(list1).intersection(list2)) > 0:
#         return True
#     else:
#         return False

In [280]:
# DiseaseDrugMatchDict = {}
# for index , row in tqdm(chem.iterrows()):
#     try:
#         normalized_components = tokenizeChemName(row[1].strip())
#     except:
#         print "Found an invalid key"
#         continue
#     print normalized_components
#     for k in diseases:
#         drugs = findDrugsForDisease(drug_association_graph,k ,0.0)
#         drugbank_list = []
#         for d in drugs:
#             drugbank_list+=drugs[d]['name'].split(' ')
            
# #         print normalized_components, drugbank_list
        
#         if len(set(drugbank_list).intersection(normalized_components)) > 0:
#             print "Matched with %s"%k
#             DiseaseDrugMatchDict[row[0]] = {}
#             if 'disease' in DiseaseDrugMatchDict[row[0]]:
#                 DiseaseDrugMatchDict[row[0]]['disease'].append(k)
#             else:
#                 DiseaseDrugMatchDict[row[0]]['disease'] = []
#                 DiseaseDrugMatchDict[row[0]]['disease'].append(k)
#             if 'chemName' not in DiseaseDrugMatchDict[row[0]]:
#                 DiseaseDrugMatchDict[row[0]]['chemName'] = row[1].strip()
                
# print DiseaseDrugMatchDict

In [703]:
import inspect
import re

def extractPostCodesDict(addrDf):
    postcodeDict = {}
    for index,row in addrDf.iterrows():
        try:
            postcodeDict[row[1]] = row[7].strip()
        except:
            print "Found invalid entry"
    return postcodeDict

def checkIndex(index):
    if index%100 == 0:
        return True
    else:
        return False

def getPC(key , postcodeDict):
    codes = []
    for k in key:
        if k in postcodeDict:
            codes.append(postcodeDict[k])
        else:
            codes.append('')
    return pd.Series(codes,index=key.index)

def getPostcode(df,postcodeDict):
    df[10] = ''
    df[10] = df.groupby(2)[2].apply(getPC,postcodeDict)   
    return df

def getDrugFamily(key, diseaseMap):
    found = 'N/A'
    for dcode in diseaseMap: 
        if key.name.find(dcode) == 0:
            found = dcode
            break
    drug = [found]*len(key)
    return pd.Series(drug,index=key.index)

def getDisease(key, diseaseMap):
    found = 'N/A'
    for dcode in diseaseMap: 
        if key.name.find(dcode) == 0:
            found = diseaseMap[dcode]['disease'].replace('\"','').replace('+',' ')
            break
    drug = [found]*len(key)
    return pd.Series(drug,index=key.index)

def getDrug(key, diseaseMap):
    found = 'N/A'
    for dcode in diseaseMap: 
        if key.name.find(dcode) == 0:
            found = diseaseMap[dcode]['chemName']
            break
    drug = [found]*len(key)
    return pd.Series(drug,index=key.index)

def getDrugPotency(key):
    found = -1
    weight = re.findall(r'[0-9]+', key.name) 
    if len(weight) > 0:
        found = int(weight[-1])
    weight = [found]*len(key)
    return pd.Series(weight,index=key.index)

def countSpecificDrugs(Df, drugs,GPs):
    df_slice = Df.groupby(3)[3].apply(getDrug,drugs)
    selected = df_slice[df_slice!='N/A']
    len(selected)
    df_selected =  Df.iloc[selected.index,:]
    df_selected = df_selected[df_selected[2].isin(GPs.keys())]
    return np.sum(df_selected[5])


def countSpecificDrugCosts(Df, drugs,GPs):
    df_slice = Df.groupby(3)[3].apply(getDrug,drugs)
    selected = df_slice[df_slice!='N/A']
    len(selected)
    df_selected =  Df.iloc[selected.index,:]
    df_selected = df_selected[df_selected[2].isin(GPs.keys())]
    return np.sum(df_selected[6])

def countDrugsByCategoryList(pdp,codes):
    total_drugs = 0.0
    for name , group in pdp.groupby(3):
        for dcode in codes:
            if name.find(dcode) == 0:
                total_drugs+=np.sum(group[5])
    return total_drugs


def countDrugsCostByCategoryList(pdp,codes):
    total_drugs = 0.0
    for name , group in pdp.groupby(3):
        for dcode in codes:
            if name.find(dcode) == 0:
                total_drugs+=np.sum(group[6])
    return total_drugs


def doImportantMappings(Df, diseaseMap , symptomMap):
    #BNF family
    Df[11] = ''  
    Df[11] = Df.groupby(3)[3].apply(getDrugFamily,symptomMap)
    #Disease
    Df[12] = ''  
    Df[12] = Df.groupby(3)[3].apply(getDisease,diseaseMap)
    #Symptom
    Df[13] = ''  
    Df[13] = Df.groupby(3)[3].apply(getDisease,symptomMap)
    #Checm Name
    Df[14] = ''  
    Df[14] = Df.groupby(3)[3].apply(getDrug,symptomMap)
    #Chem potency
    Df[15] = ''  
    Df[15] = Df.groupby(3).apply(getDrugPotency)
    return Df
                    

In [ ]:
.groupby(3).apply(getDrugPotency)

In [704]:
len(chem)

3432

In [705]:
files

[u'../BL_Work/openPrescribe/uncompressed/T201010PDPI BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201902PDPI BNFT.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201902CHEM SUBS.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201008ADDR BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201601CHEM SUBS.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201601PDPI BNFT.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201011PDPI BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201901ADDR BNFT.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201009PDPI BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201011ADDR BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201010ADDR BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201010CHEM SUBS.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201602ADDR BNFT.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201603PDPI BNFT.CSV.tar.gz',
 u'../

In [706]:
pdp = readPrescriptions('201602',files)
len(pdp)

10037914

In [707]:
%store -r Open_filtered_GPs

In [708]:
pdp_filtered = pdp[pdp[2].isin(Open_filtered_GPs.keys())].copy(deep=True)

In [709]:
len(pdp_filtered)

9062745

In [710]:
print countDrugsByCategoryList(pdp_filtered ,opiod_BNF['BNF_code'] )

1822372.0


In [711]:
print float(countDrugsCostByCategoryList(pdp_filtered ,opiod_BNF['BNF_code'] ))/56938269.0

0.397374016762


In [712]:
print countDrugsByCategoryList(pdp,antiBio_BNF['BNF_code'] )

3159501.0


In [713]:
3159501.0/56938269.0

0.0554899377078007

In [714]:
print float(countDrugsCostByCategoryList(pdp_filtered ,antiBio_BNF['BNF_code'] ))

16819395.0


In [715]:
metforminDrugs = countSpecificDrugs(pdp,diabetes_metformin,Open_filtered_GPs)

In [716]:
all_diabetes_drugs = countSpecificDrugs(pdp,diabetes,Open_filtered_GPs)

In [717]:
countSpecificDrugCosts(pdp,diabetes,Open_filtered_GPs)

53299808.029999994

In [718]:
56262126.0/3514534.0

16.00841704760859

In [719]:
all_diabetes_drugs/3514534.0 * 12

10.502512139589488

In [720]:
float(metforminDrugs)/float(all_diabetes_drugs)

0.49933792876549155

In [721]:
addr = readAddress('201601',files)
len(addr)

9905

In [722]:
addr.head()

,0,1,2,3,4,5,6,7
0,201601.0,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
1,201601.0,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
2,201601.0,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,,MIDDLESBROUGH,CLEVELAND,TS1 3BE
3,201601.0,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,,TS14 7DJ
4,201601.0,A81006,TENNANT STREET MEDICAL PRACTICE,TENNANT ST MED PRACT,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AT


In [723]:
postcodes = extractPostCodesDict(addr)

Found invalid entry


In [724]:
pdp = getPostcode(pdp , postcodes)

In [725]:
pdp = pdp.dropna()

In [726]:
pdp.tail()

,0,1,2,3,4,5,6,7,8,9,10
10037908,Q70,99M,Y04323,20030600114,Lyofoam Max 10cm x 10cm Wound Dress Poly,1.0,5.70,5.28,5.0,201602.0,KT16 0QA
10037909,Q70,99M,Y04323,20030600492,Kendall Foam 10cm x 10cm Wound Dress Pol,1.0,5.30,4.90,5.0,201602.0,KT16 0QA
10037910,Q70,99M,Y04634,0702010F0BCAAAA,Ovestin_Crm 1mg + Applic,1.0,4.45,4.13,15.0,201602.0,GU14 8QB
10037911,Q70,99M,Y04634,0703023L0BBAAAA,Mirena_Intra-Uterine System,3.0,264.00,244.35,3.0,201602.0,GU14 8QB
10037912,Q70,99M,Y04634,0703023L0BCAAAB,Jaydess_Intra-Uterine Device 13.5mg,1.0,69.22,64.07,1.0,201602.0,GU14 8QB


In [727]:
%store -r taxonomyDict

In [728]:
taxonomyDict

{'0702020Z0': {'BNF_Classification': '7.2.2',
  'ChemName': 'Dequalinium Chloride',
  'category': 'Vaginal and Vulval Infections',
  'disease': ''},
 '0202080B0': {'BNF_Classification': '2.2.8',
  'ChemName': 'Bendroflumethiazide/Potassium',
  'category': 'Diuretics With Potassium',
  'disease': ''},
 '0102000L0': {'BNF_Classification': '1.2',
  'ChemName': 'Glycopyrronium Bromide',
  'category': 'Antispasmod.&Other Drgs Alt.Gut Motility',
  'disease': ''},
 '0205040S0': {'BNF_Classification': '2.5.4',
  'ChemName': 'Prazosin Hydrochloride',
  'category': 'Alpha-Adrenoceptor Blocking Drugs',
  'disease': ''},
 '1002010Q0': {'BNF_Classification': '10.2.1',
  'ChemName': 'Pyridostigmine Bromide',
  'category': "Dgs Which Enhance Neuromus'ar Transmi'on",
  'disease': ''},
 '0411000E0': {'BNF_Classification': '4.11',
  'ChemName': 'Rivastigmine',
  'category': 'Dementia',
  'disease': ''},
 '0205051E0': {'BNF_Classification': '2.5.5',
  'ChemName': 'Cilazapril',
  'category': 'Renin-Angiot

In [729]:
pdp = doImportantMappings(pdp,DiseaseDrugs,symptoms_matched)

In [730]:
pdp.tail(n=1000)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
10036913,Q70,99M,J82630,0206020T0BDAEAH,Securon Sr_Tab 240mg,1.0,11.10,10.28,56.0,201602.0,GU14 6DH,N/A,N/A,N/A,N/A,0
10036914,Q70,99M,J82630,0206020Z0AAAAAA,Amlodipine/Valsartan_Tab 5mg/80mg,2.0,80.44,74.47,112.0,201602.0,GU14 6DH,N/A,N/A,N/A,N/A,0
10036915,Q70,99M,J82630,0206030N0AAAAAA,Nicorandil_Tab 10mg,1.0,2.90,2.70,60.0,201602.0,GU14 6DH,N/A,N/A,N/A,N/A,0
10036916,Q70,99M,J82630,0206030N0AAABAB,Nicorandil_Tab 20mg,1.0,5.79,5.37,60.0,201602.0,GU14 6DH,N/A,N/A,N/A,N/A,0
10036917,Q70,99M,J82630,0208020V0AAAAAA,Warfarin Sod_Tab 1mg,7.0,10.08,9.41,336.0,201602.0,GU14 6DH,N/A,N/A,N/A,N/A,0
10036918,Q70,99M,J82630,0208020V0AAABAB,Warfarin Sod_Tab 3mg,11.0,19.20,18.01,604.0,201602.0,GU14 6DH,N/A,N/A,N/A,N/A,0
10036919,Q70,99M,J82630,0208020V0AAADAD,Warfarin Sod_Tab 5mg,3.0,8.24,7.76,248.0,201602.0,GU14 6DH,N/A,N/A,N/A,N/A,0
10036920,Q70,99M,J82630,0208020V0AAASAS,Warfarin Sod_Tab 500mcg,2.0,3.34,3.12,56.0,201602.0,GU14 6DH,N/A,N/A,N/A,N/A,0
10036921,Q70,99M,J82630,0208020X0AAABAB,Dabigatran Etexilate_Cap 110mg,4.0,47.60,44.50,56.0,201602.0,GU14 6DH,N/A,N/A,N/A,N/A,0
10036922,Q70,99M,J82630,0208020Y0AAACAC,Rivaroxaban_Tab 20mg,2.0,151.20,139.95,84.0,201602.0,GU14 6DH,N/A,N/A,N/A,N/A,0


In [452]:
pdp.to_csv('../BL_Work/openPrescribe/serialized/' + '201601' + '.gz',compression='gzip')

In [460]:
files

[u'../BL_Work/openPrescribe/uncompressed/T201010PDPI BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201902PDPI BNFT.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201902CHEM SUBS.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201008ADDR BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201601CHEM SUBS.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201601PDPI BNFT.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201011PDPI BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201901ADDR BNFT.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201009PDPI BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201011ADDR BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201010ADDR BNFT.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201010CHEM SUBS.CSV.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201602ADDR BNFT.csv.tar.gz',
 u'../BL_Work/openPrescribe/uncompressed/T201603PDPI BNFT.CSV.tar.gz',
 u'../

In [461]:
monthList = ['201601','201602','201603']
for month in tqdm(monthList):
    pdp = readPrescriptions(month,files)
    pdp = pdp.dropna()
    pdp = getPostcode(pdp , postcodes)
    pdp = doImportantMappings(pdp,DiseaseDrugs,symptoms_matched)
    pdp.to_csv('../BL_Work/openPrescribe/serialized/' + month + '.gz',compression='gzip')


  0%|          | 0/3 [00:00<?, ?it/s]
Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/local/lib/python2.7/dist-packages/tqdm/_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python2.7/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration


100%|██████████| 3/3 [14:08<00:00, 282.73s/it]
